## Khai báo thư viện

In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

## Nạp dữ liệu

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [3]:
test_df.shape

(1459, 80)

In [4]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
test_df.shape

(1459, 80)

## Tiền xử lí dữ liệu

### Gộp tập train và test để xử lí đồng bộ

In [7]:
df_all = pd.concat([train_df.drop(['SalePrice', 'Id'], axis=1), test_df], axis=0).reset_index(drop=True)

### Filling mising value

In [8]:
def display_missing_data(df):
    missing = df.isna().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    percent = (missing / len(df)) * 100
    print(pd.DataFrame({'Missing Values': missing, 'Percent (%)': percent.round(2)}))

In [9]:
display_missing_data(train_df)

              Missing Values  Percent (%)
PoolQC                  1453        99.52
MiscFeature             1406        96.30
Alley                   1369        93.77
Fence                   1179        80.75
MasVnrType               872        59.73
FireplaceQu              690        47.26
LotFrontage              259        17.74
GarageType                81         5.55
GarageYrBlt               81         5.55
GarageFinish              81         5.55
GarageQual                81         5.55
GarageCond                81         5.55
BsmtExposure              38         2.60
BsmtFinType2              38         2.60
BsmtQual                  37         2.53
BsmtCond                  37         2.53
BsmtFinType1              37         2.53
MasVnrArea                 8         0.55
Electrical                 1         0.07


In [10]:
display_missing_data(test_df)

              Missing Values  Percent (%)
PoolQC                  1456        99.79
MiscFeature             1408        96.50
Alley                   1352        92.67
Fence                   1169        80.12
MasVnrType               894        61.27
FireplaceQu              730        50.03
LotFrontage              227        15.56
GarageQual                78         5.35
GarageCond                78         5.35
GarageYrBlt               78         5.35
GarageFinish              78         5.35
GarageType                76         5.21
BsmtCond                  45         3.08
BsmtQual                  44         3.02
BsmtExposure              44         3.02
BsmtFinType1              42         2.88
BsmtFinType2              42         2.88
MasVnrArea                15         1.03
MSZoning                   4         0.27
Functional                 2         0.14
BsmtFullBath               2         0.14
Utilities                  2         0.14
BsmtHalfBath               2      

In [11]:
numerical_features = df_all.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_all.select_dtypes(include=['object']).columns.tolist()


In [12]:
numerical_na_to_zero = [
    'MasVnrArea',
    'BsmtFinSF1',
    'BsmtFinSF2',
    'BsmtUnfSF',
    'TotalBsmtSF',
    'BsmtFullBath',
    'BsmtHalfBath',
    'GarageYrBlt',
    'GarageCars',
    'GarageArea',
    'WoodDeckSF',
    'OpenPorchSF',
    'EnclosedPorch',
    '3SsnPorch',
    'ScreenPorch',
    'PoolArea',
    'MiscVal'
]

categorical_na_cols = [
    'Alley',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'FireplaceQu',
    'GarageType',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence',
    'MiscFeature',
    'MasVnrType'
]

In [13]:
for col in numerical_na_to_zero:
    if col in df_all.columns:
        df_all[col].fillna(0, inplace=True)

for col in categorical_na_cols:
    if col in df_all.columns:
        df_all[col].fillna('None', inplace=True)

for col in categorical_features:
    if col in df_all.columns:
        # df_all[col].fillna(df_all[col].mode()[0], inplace=True)
        df_all[col].fillna('None', inplace=True)

for col in numerical_features:
    if col in df_all.columns:
        df_all[col].fillna(df_all[col].median(), inplace=True)

display_missing_data(df_all)

Empty DataFrame
Columns: [Missing Values, Percent (%)]
Index: []


## Feature engineering

In [14]:
qual_map = {'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}

df_all['TotalSF'] = df_all['TotalBsmtSF'] + df_all['1stFlrSF'] + df_all['2ndFlrSF'] 
df_all['TotalPorchSF'] = df_all['OpenPorchSF'] + df_all['EnclosedPorch'] + df_all['3SsnPorch'] + df_all['ScreenPorch']  # tổng diện tích hiên/ban công
df_all['TotalBath'] = df_all['FullBath'] + 0.5 * df_all['HalfBath'] + df_all['BsmtFullBath'] + 0.5 * df_all['BsmtHalfBath']  # tổng số phòng tắm
df_all['TotalRooms'] = df_all['TotRmsAbvGrd'] + df_all['FullBath'] + df_all['HalfBath']            # tổng số phòng có thể sử dụng
df_all['TotalSpace'] = df_all['TotalSF'] + df_all['GarageArea'] + df_all['TotalPorchSF']           # tổng không gian sử dụng (bao gồm garage và porch)

# df_all['OverallGrade'] = df_all['OverallQual'] * df_all['OverallCond']     # chất lượng tổng thể của ngôi nhà
# df_all['GarageGrade'] = df_all['GarageQual'].map(qual_map) * df_all['GarageCond']        # chất lượng garage
# df_all['ExterGrade'] = df_all['ExterQual'].map(qual_map) * df_all['ExterCond'].map(qual_map)           # chất lượng ngoại thất
# df_all['BsmtGrade'] = df_all['BsmtQual'].map(qual_map) * df_all['BsmtCond'].map(qual_map)              # chất lượng tầng hầm
# df_all['QualitySum'] = df_all['OverallQual'] + df_all['ExterQual'].map(qual_map) + df_all['BsmtQual'].map(qual_map) + df_all['GarageQual'].map(qual_map)  # tổng chất lượng

# df_all['OverallQual_GrLivArea'] = df_all['OverallQual'] * df_all['GrLivArea']
# df_all['OverallQual_TotalSF'] = df_all['OverallQual'] * df_all['TotalSF']
# df_all['OverallQual_GarageArea'] = df_all['OverallQual'] * df_all['GarageArea']

df_all['HasPool'] = (df_all['PoolArea'] > 0).astype(int)             # có hồ bơi không
df_all['HasFireplace'] = (df_all['Fireplaces'] > 0).astype(int)      # có lò sưởi không
df_all['HasGarage'] = (df_all['GarageArea'] > 0).astype(int)         # có garage không
df_all['HasBsmt'] = (df_all['TotalBsmtSF'] > 0).astype(int)          # có tầng hầm không
df_all['HasPorch'] = (df_all['TotalPorchSF'] > 0).astype(int)  

## Mã hóa và chuẩn hóa dữ liệu

In [15]:
numerical_cols = df_all.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_all.select_dtypes(include=['object']).columns.tolist()
len(numerical_cols), len(categorical_cols)

(47, 43)

In [16]:
train_len = len(train_df)
std = StandardScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

numerical_df_encoded = std.fit_transform(df_all[numerical_cols])
categorical_df_encoded = pd.get_dummies(df_all[categorical_cols], drop_first=True)
df_encoder = pd.concat(
    [pd.DataFrame(numerical_df_encoded, columns=numerical_cols),
     categorical_df_encoded.reset_index(drop=True)],
    axis=1
)

X = df_encoder[:train_len].copy()
x_test = df_encoder[train_len:].copy()
y = np.log1p(train_df['SalePrice'])


## Huấn luyện và đánh giá mô hình

### Baseline

In [17]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
base_models = {
    # === Linear Models ===
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=10),
    'Lasso': Lasso(alpha=0.001),
    'ElasticNet': ElasticNet(alpha=0.001, l1_ratio=0.5),
    
    # === Tree-based Models ===
    'Random Forest': RandomForestRegressor(
        n_estimators=300, max_depth=None, random_state=42
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
    ),
    
    # === Boosting Models ===
    'XGBoost': XGBRegressor(
        n_estimators=1000, learning_rate=0.05, max_depth=3,
        subsample=0.7, colsample_bytree=0.7, reg_lambda=1,
        random_state=42, objective='reg:squarederror'
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=1000, learning_rate=0.05, num_leaves=30,
        subsample=0.7, colsample_bytree=0.7,
        random_state=42, verbose=-1
    ),
    'CatBoost': CatBoostRegressor(
        iterations=1000, learning_rate=0.05, depth=6,
        random_state=42, verbose=False,
        allow_writing_files=False, train_dir=None
    ),
    
    # === Others ===
    'SVR': SVR(kernel='rbf', C=10, gamma='scale'),
    'K-Neighbors': KNeighborsRegressor(n_neighbors=5)
}

In [19]:
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

results = []

for name, model in base_models.items():
    rmses = []
    print(f"{name}")
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        # rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmses.append(rmse)
    
    mean_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    results.append({'Model': name, 'RMSE_mean': mean_rmse, 'RMSE_std': std_rmse})
    print(f"{name}: RMSE = {mean_rmse:.4f} ± {std_rmse:.4f}")

# Convert results to DataFrame và sắp xếp theo RMSE nhỏ nhất
df_results = pd.DataFrame(results).sort_values('RMSE_mean')
df_results

Linear Regression
Linear Regression: RMSE = 0.1982 ± 0.0634
Ridge
Ridge: RMSE = 0.1451 ± 0.0388
Lasso
Lasso: RMSE = 0.1474 ± 0.0422
ElasticNet
ElasticNet: RMSE = 0.1480 ± 0.0384
Random Forest
Random Forest: RMSE = 0.1415 ± 0.0182
Gradient Boosting
Gradient Boosting: RMSE = 0.1316 ± 0.0193
XGBoost
XGBoost: RMSE = 0.1299 ± 0.0155
LightGBM
LightGBM: RMSE = 0.1343 ± 0.0208
CatBoost
CatBoost: RMSE = 0.1251 ± 0.0162
SVR
SVR: RMSE = 0.1488 ± 0.0115
K-Neighbors
K-Neighbors: RMSE = 0.1712 ± 0.0162


,Model,RMSE_mean,RMSE_std
8,CatBoost,0.125091,0.016224
6,XGBoost,0.129877,0.015487
5,Gradient Boosting,0.131579,0.019280
7,LightGBM,0.134341,0.020826
4,Random Forest,0.141537,0.018187
1,Ridge,0.145066,0.038835
2,Lasso,0.147433,0.042197
3,ElasticNet,0.148022,0.038386
9,SVR,0.148838,0.011515
10,K-Neighbors,0.171239,0.016183


### Kết hợp mô hình 

In [20]:

def evaluate_weighted_average(models, weights, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmses = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # dự đoán từng model
        preds = np.zeros((len(y_val), len(models)))
        for i, model in enumerate(models):
            model.fit(X_train, y_train)
            preds[:, i] = model.predict(X_val)  # nếu y đã log1p

        # tính weighted average
        weights = np.array(weights) / np.sum(weights)
        final_pred = np.dot(preds, weights)

        rmse = np.sqrt(mean_squared_error(y_val, final_pred))
        rmses.append(rmse)

    return np.mean(rmses), np.std(rmses)

In [26]:
def predict_weighted_average(models, weights, X_test, log_transformed=True):
    """
    Dự đoán bằng Weighted Average từ nhiều mô hình đã huấn luyện.

    Parameters
    ----------
    models : list
        Danh sách các mô hình đã fit.
    weights : list hoặc np.array
        Trọng số tương ứng với từng mô hình.
    X_test : DataFrame hoặc array
        Dữ liệu cần dự đoán.
    log_transformed : bool
        Nếu True, tự động chuyển từ log1p về giá trị gốc.

    Returns
    -------
    np.array
        Dự đoán trung bình có trọng số.
    """
    weights = np.array(weights)
    if not np.isclose(weights.sum(), 1.0):
        weights = weights / weights.sum()  # chuẩn hóa trọng số

    preds = np.zeros((X_test.shape[0], len(models)))

    for i, model in enumerate(models):
        pred = model.predict(X_test)
        if log_transformed:
            pred = np.expm1(pred)
        preds[:, i] = pred

    final_pred = np.dot(preds, weights)
    return final_pred

In [ ]:
from sklearn.ensemble import StackingRegressor
stacking_models = {
    'Stacking_cb+rf+gb(Ridge)': StackingRegressor(
        estimators=[
            ('cb', base_models['CatBoost']),
            ('rf', base_models['Random Forest']),
            ('gb', base_models['Gradient Boosting'])
        ],
        final_estimator=Ridge(),
        cv=kf,
        n_jobs=-1        
    ),
    'Stacking_cb+lgb+knn+svr+rf+gb(Ridge)': StackingRegressor(
        estimators=[
            ('cb', base_models['CatBoost']),
            ('lgb', base_models['LightGBM']),
            ('rf', base_models['Random Forest']),
            ('gb', base_models['Gradient Boosting']),
            # ('lgb', base_models['LightGBM']),
            ('knn', base_models['K-Neighbors']),
            ('svr', base_models['SVR'])
        ],
        final_estimator=Ridge(),
        cv=kf,
        n_jobs=-1
    ),
    'Stacking_rf+svr+knn(Lasso)': StackingRegressor(
        estimators=[
            ('rf', base_models['Random Forest']),
            ('svr', base_models['SVR']),
            ('knn', base_models['K-Neighbors'])
        ],
        final_estimator=Lasso(),
        cv=kf,
        n_jobs=-1
    )
}

weight_models = {
    'weight_cb+xgb+lgb': {
        'models': ['CatBoost', 'XGBoost', 'LightGBM'],
        'weights': [0.35, 0.35, 0.30]  # Gần bằng nhau
    },
    'weight_lgb+xgb+gb': {
        'models': ['LightGBM', 'XGBoost', 'Gradient Boosting'],
        'weights': [0.40, 0.35, 0.25]
    },
}

In [32]:
from sklearn.base import clone
stacking_results = []
for name, model in stacking_models.items():
    rmses = []
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model_clone = clone(model)
        model_clone.fit(X_train, y_train)
        y_pred = model_clone.predict(X_val)

        # rmse = np.sqrt(-cross_val_score(model_clone, X, y, scoring="neg_mean_squared_error", cv=kf))
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        # r2 = cross_val_score(model_clone, X, y, scoring="r2", cv=kf)
        
        rmses.append(rmse)
    
    mean_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    stacking_results.append({'Model': name, 'RMSE_mean': mean_rmse, 'RMSE_std': std_rmse})
    print(f"{name}: RMSE = {mean_rmse:.4f} ± {std_rmse:.4f}")

df_stacking = pd.DataFrame(stacking_results).sort_values('RMSE_mean')
df_stacking


Stacking_cb+rf+gb(Ridge): RMSE = 0.1255 ± 0.0186
Stacking_cb+lgb+knn+svr+rf+gb(Ridge): RMSE = 0.1262 ± 0.0185
Stacking_rf+svr+knn(Lasso): RMSE = 0.3988 ± 0.0251


,Model,RMSE_mean,RMSE_std
0,Stacking_cb+rf+gb(Ridge),0.125496,0.018625
1,Stacking_cb+lgb+knn+svr+rf+gb(Ridge),0.126202,0.018547
2,Stacking_rf+svr+knn(Lasso),0.398782,0.025132


In [23]:
weighted_results = []

for name, model in weight_models.items():
    # Lấy danh sách models từ base_models
    model_list = [base_models[name] for name in model['models']]
    weights = model['weights']
    
    # In thông tin config
    print(f"  Models: {', '.join(model['models'])}")
    print(f"  Weights: {weights}")
    
    # Đánh giá bằng hàm có sẵn
    mean_rmse, std_rmse = evaluate_weighted_average(
        models=model_list,
        weights=weights,
        X=X,
        y=y,
        n_splits=5
    )
    
    weighted_results.append({
        'Model': name,
        'RMSE_mean': mean_rmse,
        'RMSE_std': std_rmse
        # 'N_Models': len(model_list)
    })

    print(f"  {name} Mean RMSE = {mean_rmse:.4f} ± {std_rmse:.4f}")


# Convert to DataFrame
df_weighted = pd.DataFrame(weighted_results).sort_values('RMSE_mean')
df_weighted

  Models: CatBoost, XGBoost, LightGBM
  Weights: [0.35, 0.35, 0.3]
  weight_cb+xgb+lgb Mean RMSE = 0.1256 ± 0.0179
  Models: LightGBM, XGBoost, Gradient Boosting
  Weights: [0.4, 0.35, 0.25]
  weight_lgbxg+gb Mean RMSE = 0.1279 ± 0.0190


,Model,RMSE_mean,RMSE_std
0,weight_cb+xgb+lgb,0.125599,0.017883
1,weight_lgbxg+gb,0.127911,0.019023


## Xuất ra file kết quả

In [ ]:
for name, model in stacking_models.items():
    try:
        print(f"\nProcessing: {name}")
        
        # Train on full data
        model.fit(X, y)
        
        # Predict
        preds_log = model.predict(x_test)
        preds = np.expm1(preds_log)
        
        # Create submission
        submission = pd.DataFrame({
            'Id': test_df['Id'],
            'SalePrice': preds
        })
        
        # Save
        safe_name = name.replace(' ', '_').replace('(', '').replace(')', '').replace('+', '_')
        filename = f'submission_{safe_name}.csv'
        submission.to_csv(filename, index=False)
        
        print(f" Saved: {filename}")
        
    except Exception as e:
        print(f"  Error: {e}")



🔄 Processing: Stacking_cb+rf+gb(Ridge)
  ✅ Saved: submission_Stacking_cb_rf_gbRidge_stacking.csv

🔄 Processing: Stacking_cb+lgb+knn+svr+rf+gb(Ridge)
  ✅ Saved: submission_Stacking_cb_lgb_knn_svr_rf_gbRidge_stacking.csv

🔄 Processing: Stacking_rf+svr+knn(Lasso)
  ✅ Saved: submission_Stacking_rf_svr_knnLasso_stacking.csv


In [ ]:
for name, model in weight_models.items():
    try:
        print(f"\n🔄 Processing: {name}")
        
        trained_models = []
        for model_name in model['models']:
            model_instance = base_models[model_name]
            model_instance.fit(X, y)
            trained_models.append(model_instance)
        
        # Predict using helper function
        preds = predict_weighted_average(
            models=trained_models,
            weights=model['weights'],
            X_test=x_test,
            log_transformed=True  # Tự động np.expm1
        )
        
        # Create submission
        submission = pd.DataFrame({
            'Id': test_df['Id'],
            'SalePrice': preds
        })
        
        # Save
        name = name.replace(' ', '_').replace('(', '').replace(')', '').replace('+', '_')
        filename = f'submission_{name}.csv'
        submission.to_csv(filename, index=False)
        
        print(f"  Saved: {filename}")
    except Exception as e:
        print(f"  Error: {e}")



🔄 Processing: weight_cb+xgb+lgb
  ✅ Saved: submission_weight_cb_xgb_lgb.csv

🔄 Processing: weight_lgbxg+gb
  ✅ Saved: submission_weight_lgbxg_gb.csv
